In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + "PC-Cost-202001-202004.xlsx"

pc_df = pd.read_excel(input_file_path, dtype=str)

In [ ]:
pc_df.columns = ['YearMonth', 'Province_Name', 'PC_Type', 'Cost']
pc_df["Cost"] = pc_df["Cost"].astype("float")

In [ ]:
pc_df = pc_df.groupby(by = ['YearMonth', 'Province_Name'])['Cost'].sum().reset_index()

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + "Province_Mapping.xlsx"

province_df = pd.read_excel(input_file_path, dtype=str)

pc_df = pd.merge(pc_df, province_df, on='Province_Name', how='left' )

In [ ]:
pc_df[pc_df['Province_Code'].isnull()]

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + "202001-05_province_brand_split.xlsx"

prov_brand_split = pd.read_excel(input_file_path, dtype=str)

prov_brand_split['Split_Ratio'] = prov_brand_split['Split_Ratio'].astype("float")

In [ ]:
pc_brand_df = pd.merge(pc_df, prov_brand_split, on=['YearMonth', 'Province_Code'], how='left' )

In [ ]:
pc_brand_df['Spending_Value'] = pc_brand_df['Cost'] * pc_brand_df['Split_Ratio']

In [ ]:
pc_brand_df = pc_brand_df[['YearMonth', 'Province_Code', 'SKU', 'Spending_Value']]

In [ ]:
pc_all_df = pc_brand_df.groupby(by = ['YearMonth', 'SKU'])['Spending_Value'].sum().reset_index()
pc_all_df['Province_Code'] = 'National'

In [ ]:
pc_all_df = pc_brand_df.append(pc_all_df, ignore_index=True)

In [ ]:
pc_all_df[pc_all_df['Province_Code'] == np.NAN]

In [ ]:
def format_yearmonth(year_month):
    return time.strftime("%Y/%m/%d", time.strptime(year_month,"%Y%m"))

def get_brand_name(brand):
    if brand == 'NC':
        return 'Nutrilon'
    elif (brand=='AC') | (brand == 'AP'):
        return "Aptamil"
    return ''

In [ ]:
pc_all_df["Date"] = pc_all_df['YearMonth'].apply(lambda x : format_yearmonth(x))

In [ ]:
pc_all_df["Brand_Name"] = pc_all_df['SKU'].apply(
        lambda x: get_brand_name(x))

pc_all_df['Cost_Type_1'] = 'PC'
pc_all_df['Cost_Type_2'] = ''
pc_all_df['Cost_Type_3'] = ''
pc_all_df['Cost_Type_4'] = ''
pc_all_df['AIP'] = 'P'

In [ ]:
pc_all_df = pc_all_df[['Date', 'Province_Code', 'Cost_Type_1', 'Cost_Type_2', 'Cost_Type_3', 'Cost_Type_4', 'AIP', 'Brand_Name', 'SKU', 'Spending_Value']]

In [ ]:
output_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + "202001-04_PC_Cost.xlsx"

pc_all_df.to_excel(output_file_path, index=False)